## Introduction

Analysis the OEE Detailed Events and Detailed Losses output from QliKSense

Import some libraries

In [2]:
import configparser

Config = configparser.ConfigParser()

Config.read('settings.cfg')

def ConfigSectionMap(section):
    dict1 = {}
    options = Config.options(section)
    for option in options:
        try:
            dict1[option] = Config.get(section, option)
            if dict1[option] == -1:
                DebugPrint("skip: %s" % option)
        except:
            print("exception on %s!" % option)
            dict1[option] = None
    return dict1


dir_local = ConfigSectionMap("SectionOne")['local']
dir_sanofi_share = ConfigSectionMap("SectionOne")['sanofi']
mydocs = ConfigSectionMap("SectionOne")['mydocs']

print ("dir_sanofi_share is %s" % (dir_sanofi_share))




dir_sanofi_share is C:/Users/mark_/Sanofi/Sanofi x McLaren sharing - General/Frankfurt sprint/SFD/


In [3]:
import pandas as pd
import numpy as np
import datetime

In [4]:
def get_col_widths(dataframe):
    # First we find the maximum length of the index column   
    idx_max = max([len(str(s)) for s in dataframe.index.values] + [len(str(dataframe.index.name))])
    # Then, we concatenate this to the max of the lengths of column name and its values for each column, left to right
    return [idx_max] + [max([len(str(s)) for s in dataframe[col].values] + [len(col)]) for col in dataframe.columns]


In [22]:
# read OEE Detailed Events report file for AL6 packaging machines
filename = 'AL6_Packaging_Detailed_Events.xlsx'
folder = 'OEE data'
path = os.path.join(dir_sanofi_share, folder, filename)
df_events = pd.read_excel(path, parse_dates=True, index_col='Start Date')

# read in OEE Detailed losses Analysis for AL6
filename = ('AL6_Packaging_Detailed_Losses.xlsx')
folder = 'OEE data'
path = os.path.join(dir_sanofi_share, folder, filename)
df_losses = pd.read_excel(path, parse_dates=True, index_col='Start Date')
df_losses.rename(columns=({'Stoppage Reason 3':'Category'}), inplace=True)
df_events.rename(columns=({'Stoppage Reason 3':'Category'}), inplace=True)


In [52]:
df_losses.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 22999 entries, 2021-04-01 06:05:43 to 2021-09-23 10:33:17
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Shift Long Name    22999 non-null  object        
 1   End Date           22999 non-null  datetime64[ns]
 2   Duration (min.)    22999 non-null  float64       
 3   Category           22999 non-null  object        
 4   Last Level Reason  22999 non-null  object        
 5   Machine            22999 non-null  object        
 6   Comments           22999 non-null  object        
 7   Creator            22999 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 2.1+ MB


Let's translate these stoppage categories!

| German | English | 
| ------ | ------- |
|Qualifizierung/Validierung (11)|Qualification/validation|    
|Sonstiges und Unbekannte (26)|Other and unknown|   
|Störungen und Fehlerbehebung (25)|Malfunctions and troubleshooting|    
|W&I (geplante Linie) (23)|W&I (planned line)|   
|Organisatorische Verluste und Wartezeiten (22)|Organizational losses and waiting times|   
|Rüstvorgänge (21)|Changeover|    
|Ungeplante Linie (14)|Unplanned Line|   
|Feier/Brückentag (12)|Celebration / long day|   
|Autonome Wartungen (24)|Autonomous maintenance|   
|vorbeugende Instandhaltung/Wartung (13)|Preventive maintenance / servicing|   
|Sonstige (15)|Other|   



In [24]:
df_losses['Category'].replace({
'Qualifizierung/Validierung (11)':'Qualification/validation', 
'Sonstiges und Unbekannte (26)':'Other and unknown',
'Störungen und Fehlerbehebung (25)':'Malfunctions and troubleshooting', 
'W&I (geplante Linie) (23)':'W&I (planned line)',
'Organisatorische Verluste und Wartezeiten (22)':'Organizational losses and waiting times',
'Rüstvorgänge (21)':'Changeover', 
'Ungeplante Linie (14)':'Unplanned Line',
'Feier/Brückentag (12)':'Celebration / long day', 
'Autonome Wartungen (24)':'Autonomous maintenance',
'vorbeugende Instandhaltung/Wartung (13)':'Preventive maintenance / servicing', 
'Sonstige (15)':'Other'}, inplace=True)

df_events['Category'].replace({
'Qualifizierung/Validierung (11)':'Qualification/validation', 
'Sonstiges und Unbekannte (26)':'Other and unknown',
'Störungen und Fehlerbehebung (25)':'Malfunctions and troubleshooting', 
'W&I (geplante Linie) (23)':'W&I (planned line)',
'Organisatorische Verluste und Wartezeiten (22)':'Organizational losses and waiting times',
'Rüstvorgänge (21)':'Changeover', 
'Ungeplante Linie (14)':'Unplanned Line',
'Feier/Brückentag (12)':'Celebration / long day', 
'Autonome Wartungen (24)':'Autonomous maintenance',
'vorbeugende Instandhaltung/Wartung (13)':'Preventive maintenance / servicing', 
'Sonstige (15)':'Other'}, inplace=True)

In [88]:
df_events['Month'] = df_events.index.strftime("%B")
df_events['Day'] = df_events.index.strftime("%d")
df_events.head()

# df_OEE.groupby(['Day', 'Month', 'Category'])[['OEE']].sum()

df_events['Month'] = df_events['Month'].astype(pd.api.types.CategoricalDtype(categories=['June', 'July','August','September'], ordered=True))


# drop the shift column and groupby day, month, category so we can pivot (otherwise have duplicate rows per day) 
df = df_events.groupby(['Day','Month','Category'])['Duration (min.)'].sum().reset_index()

pivot = df.pivot(index='Day', columns=['Category','Month'], values='Duration (min.)').style.background_gradient(cmap='Oranges').format("{:.2f}")


pivot

Category,Autonomous maintenance,Changeover,Malfunctions and troubleshooting,Minor Loss,Organizational losses and waiting times,Other and unknown,W&I (planned line),Without cause,Autonomous maintenance,Changeover,Malfunctions and troubleshooting,Minor Loss,Organizational losses and waiting times,Other and unknown,W&I (planned line),Without cause,Autonomous maintenance,Changeover,Malfunctions and troubleshooting,Minor Loss,Organizational losses and waiting times,Other and unknown,W&I (planned line),Without cause,Autonomous maintenance,Changeover,Malfunctions and troubleshooting,Minor Loss,Organizational losses and waiting times,Other and unknown,W&I (planned line),Without cause
Month,June,June,June,June,June,June,June,June,July,July,July,July,July,July,July,July,August,August,August,August,August,August,August,August,September,September,September,September,September,September,September,September
Day,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
01,8.92,0.00,29.75,111.83,0.00,0.00,0.00,36.52,85.38,0.00,97.08,100.43,0.00,0.00,0.00,22.65,15.57,0.00,0.00,6.33,0.00,0.00,0.00,0.00,13.32,0.00,263.55,62.43,0.00,4.80,0.00,45.52
02,65.98,0.00,897.10,75.90,0.00,0.00,0.00,7.35,0.00,0.00,108.25,66.65,0.00,0.00,0.00,62.12,0.00,0.00,12.23,34.23,0.00,0.00,0.00,0.00,12.40,0.00,25.77,44.45,0.00,65.07,0.00,27.82
03,77.93,0.00,63.03,133.77,0.00,0.00,0.00,9.82,0.00,0.00,1347.63,47.03,0.00,6.33,0.00,22.65,0.00,0.00,208.63,79.48,0.00,16.18,0.00,36.70,4.87,0.00,0.00,20.60,0.00,0.00,0.00,40.53
04,45.95,0.00,95.47,58.15,0.00,0.00,0.00,10.37,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,65.73,60.95,24.95,24.25,0.00,21.25,0.00,0.00,22.38,12.12,0.00,0.00,0.00,21.77
05,3.10,0.00,198.52,86.18,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,57.42,101.93,0.00,0.00,0.00,39.28,0.00,0.00,0.00,15.45,0.00,0.00,0.00,23.80
06,17.12,0.00,0.00,10.10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,195.00,37.78,0.00,0.00,0.00,12.52,0.00,0.00,34.00,53.70,0.00,0.00,0.00,8.92
07,0.00,0.00,33.98,97.58,0.00,0.00,0.00,32.37,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,38.93,0.00,118.72,89.93,0.00,0.00,0.00,12.50,27.98,0.00,194.40,78.68,0.00,0.00,0.00,34.15
08,21.10,0.00,43.62,38.40,0.00,0.00,0.00,6.78,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,37.87,11.18,24.78,0.00,0.00,13.45,18.60,58.00,50.92,57.50,57.47,34.70,0.00,15.53


In [48]:
category_order = ['Production',
'Organizational losses and waiting times', 
'Changeover', 
'Malfunctions and troubleshooting',
'Other and unknown', 
'Without cause', 
'Minor Loss', 
'Speed Loss', 
'Autonomous maintenance',
'Celebration / long day', 
'Other',
'Preventive maintenance / servicing', 
'Quality Loss',
'Unplanned Line',
'W&I (planned line)',
'Qualification/validation']

not using this one at the moment, as the categories are all over the place

This sorts the pivot by category, then month.

In [263]:
# writer = pd.ExcelWriter(dir + 'AL6_OEE_Losses_pivot.xlsx', engine='xlsxwriter')

# pivot_OEE_sorted = df_OEE.pivot(index='Day', columns=['Category','Month'], values='OEE')
# pivot_OEE_sorted.columns = pivot_OEE_sorted.columns.set_levels(
#     pd.CategoricalIndex(
#         pivot_OEE_sorted.columns.levels[0],
#         ['Production',
#         'Organizational losses and waiting times', 
#         'Changeover', 
#         'Malfunctions and troubleshooting',
#         'Other and unknown', 
#         'Without cause', 
#         'Minor Loss', 
#         'Speed Loss', 
#         'Autonomous maintenance',
#         'W&I (planned line)',
#         'Celebration / long day', 
#         'Other',
#         'Preventive maintenance / servicing', 
#         'Quality Loss',
#         'Unplanned Line',
#         'Qualification/validation'],
#         ordered=True
#     ), level=0
# )

# pivot_OEE_sorted = pivot_OEE_sorted.sort_index(axis=1).style.background_gradient(cmap='Oranges')\
#     .background_gradient(subset=['Production'], cmap='Purples')

# pivot_OEE_sorted.to_excel(writer)
# workbook = writer.book
# writer.save()

In [93]:
df_events.groupby([pd.Grouper(freq='D'), 'Machine'])['Duration (min.)'].mean().reset_index()

,Start Date,Machine,Duration (min.)
0,2021-04-01,AL6 Endverpacker,9.659524
1,2021-04-01,AL6 Etikettierer,7.683333
2,2021-04-01,AL6 Kartonierer,2.390964
3,2021-04-01,AL6 Montage,9.233333
4,2021-04-02,AL6 Endverpacker,5.154167
...,...,...,...
503,2021-09-22,AL6 Kartonierer,2.571491
504,2021-09-22,AL6 Montage,1.525000
505,2021-09-23,AL6 Etikettierer,4.431250
506,2021-09-23,AL6 Kartonierer,1.508470


In [114]:
filename = 'AL6_OEE_Detailed_Events_output.xlsx'
folder = 'IP21'
path = os.path.join(dir_local, folder, filename)
writer = pd.ExcelWriter(path, engine='xlsxwriter')
workbook = writer.book

df_events_grouped = df_events.groupby([pd.Grouper(freq='D'), 'Machine'])['Duration (min.)'].mean().reset_index()
pivot = df_events_grouped.pivot(index='Start Date', columns='Machine', values='Duration (min.)')
pivot.fillna(0, inplace=True)
pivot['Total Duration'] = pivot.sum(axis=1)
styler_pivot = pivot.style.background_gradient(cmap='Purples').format("{:.2f}")
styler_pivot.to_excel(writer, sheet_name='Detailed_Events (mins)')
workbook = writer.book
for x, width in enumerate(get_col_widths(pivot)):
    #set each column width to be 1*25 that of the max string so that it fits nicely
    worksheet.set_column(x, x, width * 1.25)    



In [116]:
for i in df.Month.unique():
    pivot_events = df.pivot(index='Day', columns=['Category','Month'], values='Duration (min.)').filter(regex=i)
    styler_events = pivot_events.style.background_gradient(cmap='Oranges').format("{:.2f}")
        # .format("{:,.2f}") # neither of these output to excel
        # .set_precision(2)  # neither of these output to excel
    
    styler_events.to_excel(writer, sheet_name=i)
    # pivot_OEE.to_excel(writer, sheet_name=i)
    workbook = writer.book
    worksheet = writer.sheets[i]
    # workbook = writer.book
    # cell_format = workbook.add.format()
    decimal = workbook.add_format({'num_format': '0.00'})
    worksheet.set_column('A:Q', 20, decimal)
#     call function get_col_widths to auto size the excel columns
    # for x, width in enumerate(get_col_widths(pivot_OEE)):
    #     #set each column width to be 1*25 that of the max string so that it fits nicely
    #     worksheet.set_column(x, x, width * 1.25)    



# workbook = writer.book
writer.save() # don't save until we've added more sheets below

In [100]:
# writer = pd.ExcelWriter(dir + 'AL6_OEE_Detailed_Events.xlsx', engine='xlsxwriter')
df_grouped = df.groupby([pd.Grouper(freq='D'), 'Machine'])['Duration (min.)'].mean().reset_index()
pivot = df_grouped.pivot(index='Start Date', columns='Machine', values='Duration (min.)')
pivot.fillna(0, inplace=True)
pivot['Total Duration'] = pivot.sum(axis=1)
styler_pivot = pivot.style.background_gradient(cmap='Purples')
styler_pivot.to_excel(writer, sheet_name='Detailed_Events (mins)')
worksheet = writer.sheets['Detailed_Events (mins)']
# workbook = writer.book
for x, width in enumerate(get_col_widths(pivot)):
    #set each column width to be 1*25 that of the max string so that it fits nicely
    worksheet.set_column(x, x, width * 1.25)    

# workbook = writer.book
# workbook.add_format()
workbook.add_format({'num_format': '0.00'})
writer.save() # only do this for the last sheet we're writing


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'RangeIndex'

In [78]:
df.groupby([pd.Grouper(freq='D'), 'Machine'])['Duration (min.)'].mean().reset_index()

,Start Date,Machine,Duration (min.)
0,2021-06-21,-,7.884553
1,2021-06-21,AL6 Endverpacker,106.766667
2,2021-06-21,AL6 Etikettierer,9.814583
3,2021-06-21,AL6 Kartonierer,1.558065
4,2021-06-21,AL6 Montage,107.720833
...,...,...,...
373,2021-09-18,AL6 Kartonierer,1.438248
374,2021-09-18,AL6 Montage,1.083333
375,2021-09-19,-,5.922222
376,2021-09-19,AL6 Endverpacker,1.616667


In [69]:
df_OEE[df_OEE['Category'] == 'Autonomous maintenance']
# pivot_OEE.stack()'
# df_OEE.pivot(index='Day', columns=['Category','Month'], values='OEE')
# pivot_OEE.stack()

,Category,OEE,Month,Day
Date,,,,
2021-06-21,Autonomous maintenance,46.296296,June,21
2021-06-22,Autonomous maintenance,2383.449074,June,22
2021-06-23,Autonomous maintenance,397.800926,June,23
2021-06-24,Autonomous maintenance,847.106481,June,24
2021-06-25,Autonomous maintenance,744.560185,June,25
...,...,...,...,...
2021-09-15,Autonomous maintenance,194.791667,September,15
2021-09-16,Autonomous maintenance,741.550926,September,16
2021-09-17,Autonomous maintenance,178.472222,September,17
